In [31]:
def build_regressor(df, columns, filed):
    # Обучили регрессор.
    X = df[df[filed].notnull()][columns]
    y = df[df[filed].notnull()][filed]
    model = LinearRegression()
    model.fit(X, y)
    return model

In [32]:
def predict_regressor(df, model, columns, filed):
    # Получили значения для пустых значений.
    X = df[df[filed].isnull()][columns]
    y = model.predict(X)
    return y

In [76]:
def emissions(data):
    # Считаем диапазон за границами которого все выброс.    
    q1 = data[:].quantile(0.25)
    q3 = data[:].quantile(0.75)
    iqr = q3 - q1
    return (q1 - 1.5 * iqr, q3 + 1.5 * iqr)

In [34]:
def build_regressor_outlier(df, columns, filed):
    # Обучили регрессор.
    X = df[columns]
    y = df[filed]
    model = LinearRegression()
    model.fit(X, y)
    return model

In [35]:
def predict_regressor_outlier(df, model, columns, filed):
    # Получили значения для пустых значений.
    X = df[columns]
    y = model.predict(X)
    return y

In [65]:
def clear_outlier(data, field, columns):
    # Обрабатываем выброс для конкретного поля.
    r_1, r_2 = emissions(data[field])
    if r_1 - r_2 != 0:
        share = round(data[(r_1 > data[field]) | (data[field] > r_2)].size/data.size*100, 2)
        print(f'{color.BOLD}"{field}"{color.END}\nОбщая доля выбросов в процентах {share}%\n')
        if share > 5:
            model = build_regressor_outlier(data[(r_1 < data[field]) & (data[field] < r_2)], columns, field)
            data.loc[(r_1 > data[field]) | (data[field] > r_2), field] = predict_regressor_outlier(data[(r_1 > data[field]) | (data[field] > r_2)], model, columns, field)
        else:
            data.loc[(r_1 > data[field]) | (data[field] > r_2), field] = data[(r_1 < data[field]) & (data[field] < r_2)][field].mode()[0]

In [75]:
def clear_outlier_all(data, fields, columns):
    # Обрабатываем выбросы для всех полей конкретной выборки
    for field in fields:
         clear_outlier(data, str(field), columns)

In [77]:
def grouped_by_target(data, field, target):
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1)
    sns.countplot(x=field, hue=target, data=data)
    plt.title(f'{color.BOLD}"{field}"{color.END} сгруппированная по целевой переменной')
    plt.legend(title='Target', loc='upper right')
    plt.show()

In [39]:
def grouped_by_target_kde(data, field, target, n=1000):
    limit_bal_with_target_s = data[[field, target]]
    limit_bal_s = limit_bal_with_target_s[field]
    limit_bal_target_0 = limit_bal_s[limit_bal_with_target_s[target] == 0]
    limit_bal_target_1 = limit_bal_s[limit_bal_with_target_s[target] == 1]
    plt.figure(figsize=(10, 10))
    sns.kdeplot(limit_bal_target_0, shade=True, label='0', color='g')
    sns.kdeplot(limit_bal_target_1, shade=True, label='1', color='r')
    plt.xlabel(field)
    plt.title(f'"{field}" сгруппированная по целевой переменной')
    plt.show()

In [40]:
def combine_into_category(df, field, target, var_1, var_2, id, significancy=0.05):
    sample = df.loc[df[field].isin([var_1, var_2]), [id, field, target]]
    table = sample.pivot_table(values=id, index=field, columns=target, aggfunc='count')
    chi2, p, dof, expected = chi2_contingency(table, correction=False)
    return p > significancy

In [41]:
def combine_into_category_table(df, field, target, id):
    # Определить значения показателя, которые можно собрать в одно значение
    values = df[field].unique()
    values_len = len(values)
    head =[' ']
    head.extend(values)
    result = [head]
    for i in range(values_len):
        row = [values[i]]
        for j in range(values_len):
            row.append(combine_into_category(df, field, target, values[i], values[j], id))
        result.append(row)
    dataToTable(result)

In [42]:
def dataToTable(data):
    # Вывод таблицы
    if len(data) == 0 :  
        display(HTML('<b>Нет данных</b>'))
        return
    dt = ''
    for i in range(len(data)):
        row = data[i]
        dt = dt + '<tr>'
        for j in range(len(row)):
            if j == 0:
                dt = dt + '<td>' + str(row[j]) + '</td>'
            elif i == j:
                dt = dt + '<td bgcolor="#CCCCFF"> </td>'
            else:
                if str(row[j]) == 'True':
                    dt = dt + '<td bgcolor="#CCFFCC" style="text-align: center; vertical-align: middle;">' + str(row[j]) + '</td>'
                elif str(row[j]) == 'False':
                    dt = dt + '<td bgcolor="#FFCCCC" style="text-align: center; vertical-align: middle;">' + str(row[j]) + '</td>'
                else:
                    dt = dt + '<td>' + str(row[j]) + '</td>'
        dt = dt + '</tr>'
            
    display(HTML('<table>' + dt + '</table>'))

In [43]:
def bar_and_qq(data, column_name):
    plt.figure(figsize=(20, 8))
    ax1 = plt.subplot(121)
    ax1.set_xlabel(column_name)
    ax1.set_ylabel('Количество')
    ax1.set_title(f'Распределение «{column_name}»')
    data.hist()
    plt.subplot(122)
    probplot(data, dist='norm', plot=plt)
    plt.show()

In [44]:
def shapiro_wilk(data, column_name, significance=0.05):
    t, pv = shapiro(data)
    if pv < significance:
        t, pv = shapiro(np.log(data))
        if pv < significance:
            print(f'Показатель «{column_name}» не соответствует нормальному распределнию.')
        else:
            print(f'Показатель «{column_name}» показатель нормализуется операцией логарифмирования.')
    else:
        print(f'Показатель «{column_name}» показатель нормально нормализован.')
    print('Для достоверности произведём визуальную оценку распределения признака, а так же построим QQ-график.')
    bar_and_qq(data, column_name)
    return significance <= pv

In [45]:
def confidence_interval_show(data, target_name, column_name, significance=0.05):
    plt.figure(figsize=(8, 5))
    sns.pointplot(x=target_name, y=column_name, data=data, capsize=.1)
    plt.title(f'Доверительный интервал ({100 - significance} %) для «{column_name}»')
    plt.show()

In [46]:
def mann_whitneyu(data, target_name, column_name, significance=0.5):
    data_with_target_s = data[[column_name, target_name]]
    data_s = data_with_target_s[column_name]
    data_target_0 = data_s[data_with_target_s[target_name] == 0]
    data_target_1 = data_s[data_with_target_s[target_name] == 1]
    t, pv = mannwhitneyu(data_target_0, data_target_1)
    if (pv < significance):
        print(f'По показателю «{column_name}» можно построить модель.')
    else:
        print(f'По показателю «{column_name}» нельзя построить модель.')
    print('Для достоверности произведём визуальную оценку доверительного интервала.')
    confidence_interval_show(data, target_name, column_name, significance)
    return pv < significance

In [47]:
def check_rate(data, target_name, column_name, show_kde=True):
    display(Markdown(f'__Оценка признака «{column_name}» в разрезе целевой переменной__'))
    if show_kde:
        grouped_by_target_kde(data, column_name, target_name, df_train.shape[0])
    shapiro_wilk(data[column_name], column_name)
    return mann_whitneyu(data, target_name, column_name)

In [48]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [49]:
def balance_df_by_target(df, target_name):
    target_counts = df[target_name].value_counts()
    major_class_name = target_counts.argmax()
    minor_class_name = target_counts.argmin()
    disbalance_coeff = int(target_counts[major_class_name] / target_counts[minor_class_name]) - 1
    for i in range(disbalance_coeff):
        sample = df[df[target_name] == minor_class_name].sample(target_counts[minor_class_name])
        df = df.append(sample, ignore_index=True)
    return df.sample(frac=1) 

In [81]:
def show_proba_calibration_plots(y_predicted_probs, y_true_labels):
    preds_with_true_labels = np.array(list(zip(y_predicted_probs, y_true_labels)))
    thresholds = []
    precisions = []
    recalls = []
    f1_scores = []
    for threshold in np.linspace(0.1, 0.9, 9):
        thresholds.append(threshold)
        precisions.append(precision_score(y_true_labels, list(map(int, y_predicted_probs > threshold))))
        recalls.append(recall_score(y_true_labels, list(map(int, y_predicted_probs > threshold))))
        f1_scores.append(f1_score(y_true_labels, list(map(int, y_predicted_probs > threshold))))
    scores_table = pd.DataFrame({'f1':f1_scores,
                                 'precision':precisions,
                                 'recall':recalls,
                                 'probability':thresholds}).sort_values('f1', ascending=False).round(3)
    figure = plt.figure(figsize = (15, 5))
    plt_calibration = figure.add_subplot(121)
    plt_calibration.plot(thresholds, precisions, label='Precision', linewidth=4)
    plt_calibration.plot(thresholds, recalls, label='Recall', linewidth=4)
    plt_calibration.plot(thresholds, f1_scores, label='F1', linewidth=4)
    plt_calibration.set_ylabel('Scores')
    plt_calibration.set_xlabel('Probability threshold')
    plt_calibration.set_title('Probabilities threshold calibration')
    plt_calibration.legend(bbox_to_anchor=(0, 1))   
    plt_calibration.table(cellText = scores_table.values,
               colLabels = scores_table.columns, 
               colLoc = 'center', cellLoc = 'center', loc = 'bottom', bbox = [0, -1.3, 1, 1])
    plt_histogram = figure.add_subplot(122)
    plt_histogram.hist(preds_with_true_labels[preds_with_true_labels[:, 1] == 0][:, 0], 
              label='Another class', color='royalblue', alpha=1)
    plt_histogram.hist(preds_with_true_labels[preds_with_true_labels[:, 1] == 1][:, 0], 
              label='Main class', color='darkcyan', alpha=0.8)
    plt_histogram.set_ylabel('Number of examples')
    plt_histogram.set_xlabel('Probabilities')
    plt_histogram.set_title('Probability histogram')
    plt_histogram.legend(bbox_to_anchor=(1, 1))
    plt.show()

In [51]:
def show_learning_curve_plot(estimator, X, y, cv=3, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, 
                                                            cv=cv, 
                                                            scoring='f1',
                                                            train_sizes=train_sizes, 
                                                            n_jobs=n_jobs)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.figure(figsize=(15,8))
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    plt.title(f"Learning curves ({type(estimator).__name__})")
    plt.xlabel("Training examples")
    plt.ylabel("Score")     
    plt.legend(loc="best")
    plt.grid()
    plt.show()

NameError: name 'np' is not defined

In [52]:
def show_feature_importances(feature_names, feature_importances, get_top=None):
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
    feature_importances = feature_importances.sort_values('importance', ascending=False)
    plt.figure(figsize = (20, len(feature_importances) * 0.355))
    sns.barplot(feature_importances['importance'], feature_importances['feature'])
    plt.xlabel('Importance')
    plt.title('Importance of features')
    plt.show()
    if get_top is not None:
        return feature_importances['feature'][:get_top].tolist()